# Notebook to select host galaxies and submit candidates to TNS

In [27]:
import sys, os
import pandas as pd
import numpy as np
import ipyaladin as ipyal
from ipywidgets import Layout, Box, widgets

In [28]:
from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, Column

In [29]:
sys.path.append("./lib")
from alerce_tns import *

In [30]:
# continuously load modules (for easier debugging)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Start API

In [31]:
alerce = alerce_tns()

# Load recent candidates

# CHANGE DATE

In [32]:
refstring = "test"
candidates_dir = "candidates"
candidates = open("%s/%s.txt" % (candidates_dir, refstring)).read().splitlines()
print(candidates)

['ZTF19acssnul', 'ZTF18aaizesz']


# Select host galaxies using aladin widget

In [14]:
alerce.select_hosts(candidates, catsHTM=False, vizier=False)

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

In [15]:
alerce.candidate_host_names

{'ZTF18aaizesz': 'LEDA 2362390', 'ZTF19acssnul': 'WISEA J003633.27-183740.6'}

# Manual modifications

In [16]:
#oid = "ZTF19acmbjll"
#alerce.candidate_host_names[oid] = "WISEA J052037.89-090209.7"
#alerce.candidate_host_redshifts[oid] = "nan"

# Clean nans

In [17]:
for k in list(alerce.candidate_host_redshifts.keys()):
    if alerce.candidate_host_redshifts[k] == "nan":
        alerce.candidate_host_redshifts.pop(k)

# Save hosts

In [12]:
df_host_names = pd.DataFrame.from_dict(alerce.candidate_host_names, orient='index', columns=["host_id"])
df_host_names.index.name='oid'
df_host_redshifts = pd.DataFrame.from_dict(alerce.candidate_host_redshifts, orient='index', columns=["z"])
df_host_redshifts.index.name='oid'
df_hosts = df_host_names.join(df_host_redshifts)
df_hosts.to_csv("candidates/%s_hosts.csv" % refstring)
df_hosts.head(100)

,host_id,z
oid,,
ZTF18aaizesz,WISEA J143834.33+500512.2,0.068077
ZTF18abolsfz,WISEA J004856.74-012702.9,0.053636
ZTF19actgcmq,WISEA J181517.05+655024.0,NaN
ZTF19actgldv,WISEA J181243.12+693024.6,NaN
ZTF19acthhzn,SDSS J014926.00-154540.1,0.109
ZTF19acthshd,WISEA J011630.37+000542.2,0.105744
ZTF19acthtdb,SDSS J004003.85+111146.4,0.129
ZTF19acthvrf,WISEA J193631.81+841435.4,NaN
ZTF19actjohz,WISEA J044614.85+044425.3,NaN


# Display host galaxy properties

In [18]:
for oid in candidates:
    if oid in alerce.candidate_host_redshifts.keys():
        print(oid, alerce.candidate_host_names[oid], alerce.candidate_host_redshifts[oid])
    elif oid in alerce.candidate_host_names.keys():
        print(oid, alerce.candidate_host_names[oid])
    else:
        alerce.candidate_host_names[oid] = ""
        alerce.candidate_host_redshifts[oid] = ""
        print(oid)

ZTF18aaizesz LEDA 2362390 0.06805
ZTF19acssnul WISEA J003633.27-183740.6 0.05332


# Sandbox or real

In [19]:
sandbox = False # True: test in sandbox, False: send to real website

# Get API key and url

In [20]:
if sandbox:
    print("Getting API sandbox key...")
    api_key = open("API_sandbox.key", "r").read()
else:
    print("Getting API key...")
    api_key = open("API.key", "r").read()
    
if sandbox:
    url_tns_api="https://sandbox-tns.weizmann.ac.il/api"
else:
    url_tns_api="https://wis-tns.weizmann.ac.il/api"
    
if not sandbox:
    print("WARNING: submitting to real (not sandbox) TNS website")

Getting API key...


# Prepare TNS report

# CHECK AUTHOR LIST!!!
(key is used to query only at this step)

In [26]:
reporter = "F. Forster, F.E. Bauer, G. Pignata, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, F.E. Bauer, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"
report = {"at_report": {}}
counter = 0
for oid in candidates:
    #if oid not in test:#["ZTF19acgjpfd", "ZTF18acszatf", "ZTF18acrkmnm"]:
    #    continue
    #if oid != "ZTF19acmxymo": #ZTF19acnqqhl":#in ["ZTF19acmdwjy", "ZTF19acmdwjz"]:
    #    continue
    print("\n", counter, oid)
    sn_report = alerce.do_TNS_report(api_key, oid, reporter, verbose=False, test=False)
    if sn_report:
        report["at_report"]["%i" % counter] = sn_report
        counter += 1


 0 ZTF18aaizesz
http://alerce.online/object/ZTF18aaizesz


SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF18aaizesz 

 1 ZTF19acssnul
http://alerce.online/object/ZTF19acssnul


Early SN candidate (g-rise > 0.32+-0.04 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acssnul 


# Display report

In [17]:
print(json.dumps(report, indent=4, sort_keys=True))

{
    "at_report": {
        "0": {
            "at_type": "1",
            "dec": {
                "error": "0.09",
                "units": "arcsec",
                "value": "-1.4508643"
            },
            "discovery_datetime": "2019-11-18 05:03:08.997",
            "groupid": "74",
            "host_name": "WISEA J004856.74-012702.9",
            "host_redshift": "0.053636",
            "internal_name": "ZTF18abolsfz",
            "non_detection": {
                "comments": "Data provided by ZTF",
                "exptime": "30",
                "filter_value": "110",
                "flux_units": "1",
                "instrument_value": "196",
                "limiting_flux": "19.8325996398926",
                "obsdate": "2019-11-15 04:31:49.002",
                "observer": "Robot"
            },
            "photometry": {
                "photometry_group": {
                    "0": {
                        "comments": "Data provided by ZTF",
                    

# Save report

In [18]:
json_report = '%s/%s.json' % (candidates_dir, refstring)
with open(json_report, 'w') as outfile:
    json.dump(report, outfile)

# Send to TNS

**WARNING: key is used to send candidates!**

In [19]:
json_report

'candidates/20191118.json'

In [20]:
r = alerce.send_json_report(api_key, url_tns_api, json_report)

# Show output

In [21]:
r.content

b'{"id_code":200,"id_message":"OK","data":{"report_id":27518,"recieved_data":{"at_report":{"0":{"ra":{"value":12.2364744,"error":0.09,"units":"arcsec"},"dec":{"value":-1.4508643,"error":0.09,"units":"arcsec"},"groupid":74,"reporter":"F. Forster, F.E. Bauer, G. Pignata, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, F.E. Bauer, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker","discovery_datetime":"2019-11-18 05:03:08.997","at_type":1,"internal_name":"ZTF18abolsfz","remarks":"Early SN candidate (r-rise > 0.10+-0.01 mag\\/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http:\\/\\/alerce.online\\/object\\/ZTF18abolsfz ","non_detection":{"obsdate":"2019-11-15 04:31:49.002","limiting_flux":19.832599639893,"flux_units":1,"filter_value":110,"instrument_value":196,"exptime":30,"observer":"Robot","comme

# Check the following [link](https://wis-tns.weizmann.ac.il/search?&discovered_period_value=1&discovered_period_units=months&unclassified_at=0&classified_sne=0&name=&name_like=0&isTNS_AT=all&public=all&ra=&decl=&radius=&coords_unit=arcsec&groupid%5B%5D=74&classifier_groupid%5B%5D=null&objtype%5B%5D=null&at_type%5B%5D=null&date_start%5Bdate%5D=&date_end%5Bdate%5D=&discovery_mag_min=&discovery_mag_max=&internal_name=&discoverer=&classifier=&spectra_count=&redshift_min=&redshift_max=&hostname=&ext_catid=&ra_range_min=&ra_range_max=&decl_range_min=&decl_range_max=&discovery_instrument%5B%5D=null&classification_instrument%5B%5D=null&associated_groups%5B%5D=null&at_rep_remarks=&class_rep_remarks=&num_page=50) and confirm that the latest SNe are from the reported day

https://wis-tns.weizmann.ac.il/search